# Лабороторная работа 2.
# Рак Алексей. Группа 3. Курс 4

# Условие №6.8, стр.179
По 1000 реализаций оценить характеристики наращенной суммы $С$: $C_{min}$, $C_{max}$, $E\{C\}$, $D\{C\}$, $P\{13000 \leq C \leq 13200\}$

$n_1 = n_2 = 2$ года

$P = 10000$

$i_1 = R(9\%, 10.2\%)$

$i_2 = Tr(8\%, 12\%; 9\%, 10\%)$

$h_i = 2.6 + 0.4 i + \epsilon_i$

$\epsilon_i = x + \eta$

$\eta: P(\eta = -0.1) = 0.1, P(\eta = 0.1) = 0.2, P(\eta = 0) = 0.7$

# Ход работы

Ход работы. Смоделируем базовую случайную величину α с использованием мультипликативного конгруэнтного метода:

$\alpha_i = \alpha_i ' / M$

$\alpha_i ' = \beta \alpha\_{i - 1} ' mod M$

$i = 1, 2, ...$

$\alpha_0 ' = \beta = 65539$

$M = 2147483648$

Методом обратной функции получим формулы для равномерного и трапециидального распределения:

$R(a, b) = y (b - a) + a$

$Tr(a, b, c, d) = \begin{cases}
a + \sqrt{y (c - a) (b + d - a - c)}, y \leq \frac{c - a}{b + d - a - c}\\
\frac{y(b + d - a - c) + a + c}{2}, \frac{c - a}{b + d - a - c} < y \leq 1 - \frac{b - d}{b + d - a - c}\\
b - \sqrt{(1 - y)(b - d)(b + d - a - c)}, 1 - \frac{b - d}{b + d - a - c} < y
\end{cases}$

Получим наращенную сумму без учёта инфляции по следующей формуле:
$S = P(1 + i_1)^{n_1}(1 + i_2)^{n_2}$

Посчитаем инфляцию по формуле:
$j = \prod\limits_{i = 1}^{16}\left(1 + \frac{h_i}{100}\right)$

Итоговую наращенную сумму получим по формуле:
$C = \frac{S}{j}$

# Реализация

In [99]:
import numpy as np

In [2]:
class BRV:
    def __init__(self):
        self.alpha = 65539
        self.beta = 65539
        self.M = 2147483648
        
    def __call__(self):
        self.alpha = (self.alpha * self.beta) % self.M
        return self.alpha / self.M

In [3]:
class Uniform(BRV):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
    def __call__(self):
        return super().__call__() * (self.b - self.a) + self.a

In [8]:
class Tr(BRV):
    def __init__(self, a, b, c, d):
        super().__init__()
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.density = self.b + self.d - self.a - self.c
        
    def __call__(self):
        brv = super().__call__()
        if brv < (self.c - self.a) / self.density:
            D = np.sqrt(brv * (self.c - self.a) * self.density)
            return self.a + D
        if brv > 1 - (self.b - self.d) / self.density:
            D = np.sqrt((1 - brv) * (self.b - self.d) * self.density)
            return self.b - D
        return (brv * self.density + self.a + self.c) / 2 

In [16]:
class Discrete(BRV):
    def __init__(self, a, b, c, ap, bp):
        super().__init__()
        self.a = a
        self.b = b
        self.c = c
        self.ap = ap
        self.bp = bp + ap
        
    def __call__(self):
        brv = super().__call__()
        if brv < self.ap:
            return self.a
        if brv < self.bp:
            return self.b
        return self.c

In [102]:
P = 10000
n1 = 2
n2 = 2
random1 = Uniform(9, 10.2)
random2 = Tr(8, 12, 9, 10)
ksir = Discrete(-0.1, 0.1, 0, 0.1, 0.2)
Cs = []
for it in range(10000):
    i1 = random1()
    i2 = random2()
    S = P * ((1 + i1 / 100) ** n1) * ((1 + i2 / 100) ** n2)
    h = 3
    ksi = 0
    j = 1
    for it2 in range(16):
        j *= (1 + (h + ksi) / 100)
        ksi += ksir()
        h += 0.4
    C = S / j
    Cs.append(C)
Cs = np.array(Cs)

# Результаты

In [103]:
print("C_min = {}".format(Cs.min()))
print("C_max = {}".format(Cs.max()))
print("E{C} =", Cs.mean())
print("D{C} =", np.sqrt(np.mean((Cs - Cs.mean()) ** 2)))
print("P{13000 <= C <= 13200} =", np.sum(np.logical_and(13000 <= Cs, Cs <= 13200)))

C_min = 5145.892602521651
C_max = 6169.934234991363
E{C} = 5653.221838590885
D{C} = 159.84844413271668
P{13000 <= C <= 13200} = 0


# Оценка результата


При моделировании наращенной суммы с данными параметрами получили следующие результаты:
- наименьшая наращенная сумма равна 5145
- наибольшая наращенная сумма равна 6129
- средняя наращенная сумма на 1000 реализаций равна 5655
- дисперсия наращенной суммы на 1000 реализаций равна 159
- ни при одной из 1000 реализаций наращенная сумма не попала в промежуток $[13000, 13200]$